In [16]:
import pandas as pd
from pandas import read_excel
import numpy as np
import re
from re import sub
import multiprocessing
from unidecode import unidecode
import os
from time import time 
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Activation,Embedding,Flatten,Bidirectional,MaxPooling2D, Conv1D, MaxPooling1D
from keras.optimizers import SGD,Adam
from keras import regularizers
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import h5py
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, Bidirectional, LSTM, Dropout, Dense, Flatten,GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
import matplotlib.pyplot as plt
from itertools import chain
from sklearn.neighbors import KNeighborsClassifier

from imblearn.over_sampling import SMOTE

In [2]:
file_path = 'corpus_all.txt'  # Replace with the actual path to your text file

lines = []  # Empty list to store the lines

try:
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            lines.append(line.strip())  # Append the line to the list, removing any leading/trailing whitespace

except FileNotFoundError:
    print("File not found. Please check the file path and try again.")

#file_path = 'corpus_all.txt'  # Replace with the actual path to your text file
first_words = []  # Empty list to store the first words

try:
    for line in lines:
        words = line.strip().split()  # Split the line into words
        if words:
            first_word = words[0]  # Get the first word
            first_words.append(first_word)  # Append the first word to the list

except FileNotFoundError:
    print("File not found. Please check the file path and try again.")


# Define the status levels
status_levels = ["happy","angry","fear","disgust","sad","surprise",]

# Initialize a list to store the extracted status levels and sentences
extracted_data = []

# Iterate over the texts
for text in lines:
    # Initialize variables to store the extracted status level and sentence
    status = ""
    sentence = ""

    # Iterate over the status levels
    for level in status_levels:
        if level in text:
            # Split the text based on the status level
            split_text = text.split(level, 1)

            # Extract the status level and sentence
            status = level
            sentence = split_text[1].strip()
            break

    # Append the extracted status level and sentence to the list
    extracted_data.append({"Status": status, "Sentence": sentence})
    
df = pd.DataFrame(extracted_data)

In [36]:
df

,Status,Sentence
0,sad,"বরাবর, মাননীয় প্রধানমন্ত্রী গণপ্রজাতন্ত্রী বা..."
1,sad,তদন্ত করে লাভ কী ? কোন দিন কোন তদন্তের পর কিছু...
2,angry,দেশকি মধ্যম আয়ের দেশে রুপান্তর হচ্ছে নাকি মগের...
3,happy,আমি গর্বিত আমি মুসলিম। আমি সংগ্রামী। আমি যোদ্ধ...
4,angry,রাতের বেলা আবার কিসের সকাল।
...,...,...
6284,surprise,ভারতে প্রতিদিন ১৮ টা রেপ কেইস এন্ট্রি হয়
6285,surprise,এই কি স্বাধীন দেশ
6286,sad,স্বাধীন দেশের নাগরিক হয়ে ও আজ আমরা পরাধীন।
6287,angry,চুদলাম দাদা পয়সা নাই


In [37]:
X = df['Sentence']

In [38]:
# Map sentiment labels to numerical values
label_map  = {'happy': 0, 'angry': 1, 'fear': 2, 'disgust': 3, 'sad': 4, 'surprise': 5}
y = df['Status'].map(label_map)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [40]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust the number of features as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = tfidf_vectorizer.transform(X_test).toarray()


In [42]:
# print(X_train, X_test, y_train, y_test)
print(X_train_tfidf, X_test_tfidf)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [12]:
k = 5  # Number of neighbors (you can choose a different value)
knn_classifier = KNeighborsClassifier(n_neighbors=k)
knn_classifier.fit(X_train_tfidf, y_train)


KNeighborsClassifier()

In [13]:
y_pred = knn_classifier.predict(X_test_tfidf)


In [14]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print(classification_report(y_test, y_pred))

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


Accuracy: 0.35135135135135137
              precision    recall  f1-score   support

           0       0.35      0.91      0.50       391
           1       0.35      0.17      0.23       268
           2       0.25      0.02      0.03        66
           3       0.21      0.02      0.04       141
           4       0.45      0.11      0.18       278
           5       0.39      0.06      0.11       114

    accuracy                           0.35      1258
   macro avg       0.33      0.21      0.18      1258
weighted avg       0.35      0.35      0.26      1258

Confusion Matrix:
[[355  18   2   1  11   4]
 [204  45   0   3  11   5]
 [ 53   4   1   1   6   1]
 [111  22   0   3   5   0]
 [217  24   1   4  31   1]
 [ 85  15   0   2   5   7]]


# Apply SMOTE

In [45]:
# Assuming you have loaded your dataset into a pandas DataFrame named 'df'
# X should contain the features (in this case, the 'Sentence' column) and y should contain the target class labels (the 'Status' column)
X = df['Sentence']
y = df['Status']

# Create a TfidfVectorizer to convert text to numerical features
vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(X)

# Create an instance of the SMOTE class
smote = SMOTE(sampling_strategy='auto', random_state=42)

# Apply SMOTE to generate synthetic samples for the minority class (fear)
X_resampled, y_resampled = smote.fit_resample(X_vectorized, y)

# Convert the resampled data back to a DataFrame
dataframe = pd.DataFrame({'Status': y_resampled, 'Sentence': X_resampled})

In [73]:
dataframe['Sentence']

0          (0, 3085)\t0.04325065094600315\n  (0, 2863)\...
1          (0, 1527)\t0.13503058341724922\n  (0, 3140)\...
2          (0, 2695)\t0.36675405908123854\n  (0, 3283)\...
3          (0, 1078)\t0.3174361850159315\n  (0, 1700)\t...
4          (0, 3139)\t0.815637800513949\n  (0, 1157)\t0...
                               ...                        
11425      (0, 3221)\t0.17982368306812407\n  (0, 2893)\...
11426      (0, 2844)\t0.28862964104909306\n  (0, 1868)\...
11427      (0, 1543)\t0.23329132119279034\n  (0, 1097)\...
11428      (0, 3314)\t0.020292737135870875\n  (0, 2655)...
11429      (0, 1165)\t0.10130664558951535\n  (0, 1147)\...
Name: Sentence, Length: 11430, dtype: object

In [47]:
X_new = dataframe['Sentence']

In [48]:
y_new = dataframe['Status'].map(label_map)

In [49]:
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

In [67]:


tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust the number of features as needed
X_train_tfidf_new = tfidf_vectorizer.fit_transform(X_train_new).toarray()
X_test_tfidf_new = tfidf_vectorizer.transform(X_test_new).toarray()



AttributeError: lower not found

In [35]:
k = 5  # Number of neighbors (you can choose a different value)
knn_classifier = KNeighborsClassifier(n_neighbors=k)
knn_classifier.fit(X_train, y_train)


ValueError: setting an array element with a sequence.